In [1]:
import pandas as pd
covid_data = pd.read_csv('data\covid_data.csv')
country_vaccinations = pd.read_csv('data\country_vaccinations.csv')

In [2]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [3]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [4]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [5]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [8]:
country_vaccinations['date'] = pd.to_datetime(country_vaccinations['date'])

In [37]:
filter = (country_vaccinations['date'] >= covid_data['date'].min()) & (country_vaccinations['date'] <= covid_data['date'].max())
covid_df = covid_data.merge(country_vaccinations[filter], how='left', on=['country', 'date'])
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 0 to 86784
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 86785 non-null  datetime64[ns]
 1   country                              86785 non-null  object        
 2   confirmed                            86785 non-null  float64       
 3   deaths                               86785 non-null  float64       
 4   recovered                            86785 non-null  float64       
 5   active                               86785 non-null  float64       
 6   daily_confirmed                      86564 non-null  float64       
 7   daily_deaths                         86564 non-null  float64       
 8   daily_recovered                      86564 non-null  float64       
 9   iso_code                             18451 non-null  object        
 10  total_vacc

In [38]:
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

In [42]:
filter = covid_df['country'] == 'Russia'
covid_df[filter]['recover_rate'].mean()

67.06273489477655

In [43]:
covid_df.to_csv('data\covid_df.csv')